<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/2021notebooks/2021_0903BERT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ごく簡単な BERT の使い方

- date: 2021_0903
- author: 浅川伸一
- 参考サイト: https://huggingface.co/bert-base-multilingual-cased


In [ ]:
# MeCab, fugashi, ipadic のインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

!pip install fugashi[unidic]
!python -m unidic download
!pip install ipadic

In [ ]:
# allennlp の transformers のインストール
!pip install transformers
from transformers import BertTokenizer, BertModel
from transformers import pipeline

In [ ]:
# 学習済 BERT 日本語モデルのインストール special thanks to 東北大学 乾研究室
unmasker = pipeline('fill-mask', model='cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
# ここからは，実演です
masked_sentences = [['ニューラルネットワーク は 神経 心理学 の [MASK] です.'],
                    ['札幌 クラーク 病院 は 神経 心理学 の [MASK] です.'],
                    ['東京女子大学 は [MASK] な 大学 です.'],
                    ['[MASK] 大学 は リベラル な 大学 です.'],
                    ]
for sentence in masked_sentences:
    for x in unmasker(sentence):
        print(f"{x['sequence']}, スコア: {x['score']:.3f} ")

#for x in unmasker("ニューラルネットワーク は 神経 心理学 の [MASK] です."):
#    print(f"{x['sequence']}, スコア:{x['score']:.3f} ")
#print(unmasker("札幌 クラーク 病院 は 神経 心理学 の [MASK] です."))

In [ ]:
unmasker(['CCAP は 神経 心理学 に とって [MASK] な 存在 です.'])

In [ ]:
# 以下は BERT モデルの内部を確認する作業
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
#print(model)

In [ ]:
from transformers import BertConfig

# 東北大学 乾研究室の設定を確認
config_japanese = BertConfig.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
print(config_japanese)

In [ ]:
# 分かち書きをするための tokenizer を呼び出して利用するため
import torch 
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


In [ ]:
input_ids = tokenizer.encode(f'''
    青葉山で{tokenizer.mask_token}の研究をしています。
''', return_tensors='pt')
print(input_ids)